In [ ]:
 ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

1. CV_TM_CCOEFF
    * 去掉直流成份的相關係數，越大越相似
2. CV_TM_CCOEFF_NORMED
    * 正規化 去掉直流成份的相關係數
    * 保證當 pixel 亮度都乘上同一係數時，相似度不變
    * 計算出的相關係數被限制在了 -1 到 1 之間
3. CV_TM_CCORR
    * 相關係數，越大越相似
4. CV_TM_CCORR_NORMED
    * 正規化相關係數，越大越相似
    * 保證當 pixel 亮度都乘上同一係數時，相似度不變
5. CV_TM_SQDIFF
    * 平方差，越小越相似
6. CV_TM_SQDIFF_NORMED
    * 正規化平方差，越小越相似
    * 保證當 pixel 亮度都乘上同一係數時，相似度不變

CV_TM_CCOEFF,CV_TM_CCOEFF_NORMED,CV_TM_SQDIFF三種方式都可以找到正確的人頭!

###### %%writefile test
## Practice
1. Input images from video file WiiPlay.mp4 with frame number between 4820 and 5000 (i.e., level 15).
2. Acquire a face template from the first frame (frame number = 4820).
3. Use <i>cv2.matchTemplate()</i> to perform template matching on subsequent frames.
4. Draw a blue rectangle around each good match and show the output images.
5. (Optional) Use <i>cv2.minMaxLoc()</i> to find the best match and draw a red rectangle around the best match.
6. Observe which method is better ('cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED') 
7. Upload your Jupyter code file (*.ipynb)

In [218]:
import cv2
import numpy as np

cap = cv2.VideoCapture('WiiPlay.mp4')

# 檢查影片檔案是否正確打開
if not cap.isOpened():
    raise IOError("Cannot open the video file")

# 設置開始、結束時間
# Set start and end time
start_time = 4821
end_time = 5000
frame_seq = 4820 # 計算開始的影格 # Calculate the starting frame
out_size = (640, 360)

# 使用 X264 編碼
# Use X264 encoding

#fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# 建立 VideoWriter 物件，輸出影片至 output.mp4
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, out_size)

# 第一個參數標誌 CAP_PROP_POS_FRAMES 是下一個要解碼/捕獲的幀的從 0 開始的索引。
# 第二個參數定義幀數，範圍從 0 到 (time_length*fps - 1)。
cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq);
# 建立計算影格的計數器
counter = start_time 

select = False
mouse_pressed = False  # 設定flag: 有沒有按下滑鼠
y0 = x0 = w = h = 0 #168 232 292 350

ret, frame = cap.read()
cur_frame = cv2.resize(frame, out_size, 0, 0, interpolation=cv2.INTER_AREA)
show_img = np.copy(cur_frame)


template = np.copy(show_img[175:222, 301:339])
#template = np.copy(show_img[y0:y0+h, x0:x0+w]) # 新的影像

cv2.imshow('template',template)
# cv2.waitKey()
# cv2.destroyAllWindows()
#template = cv2.resize(template, (55,60), 0, 0, interpolation=cv2.INTER_AREA)
h,w = template.shape[:2]
print(template.shape)
print(w,h)
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
show_img = np.copy(cur_frame)

# 選擇使用的 methods
select = 0


flage = 0
while True:
    ret, img = cap.read()
    img = cv2.resize(img, out_size, 0, 0, interpolation=cv2.INTER_AREA)
    cv2.imshow('tm', show_img)
    # 判斷結束時間
    if ret == False  or counter >= end_time:
        break
    else:
        counter += 1
    k = cv2.waitKey(1)
    if k == -1:
        k = select
    else:
        if k == 27:  #ESC鍵
            break
        k = int(chr(k))
        select = k
    if k == 27:  #ESC鍵
        break
    elif k >= 0:
        index = k
        if 0 <= index < len(methods):
            method = methods[index]
            res = cv2.matchTemplate(img, template, eval(method))
            
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
            if eval(method) in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                top_left = (min_loc[0], min_loc[1])
            else:
                top_left = (max_loc[0], max_loc[1])
            bottom_right = (top_left[0] + w, top_left[1] + h)
            
            res = cv2.normalize(res, None, 0, 1, cv2.NORM_MINMAX)      #正規化到0~1區間 
            res_img = np.copy(res)
            res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2BGR).astype(np.uint8)
            
            if eval(method) in [cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED]:
                flage = 1
                val,result = cv2.threshold(res,0.95,1.0,cv2.THRESH_BINARY)
                cv2.imshow('result',result)
                match_locs = cv2.findNonZero(result)
            elif eval(method) in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                flage = 1
                val,result = cv2.threshold(res,0.01,1.0,cv2.THRESH_BINARY_INV)
                cv2.imshow('res',res)
                cv2.imshow('result',result)
                match_locs = cv2.findNonZero(result)
            else:
                cv2.imshow('res',res)
                flage = 0
                for y in range(0, res.shape[0]):
                    for x in range(0, res.shape[1]):
                        if (index >= methods.index('cv2.TM_SQDIFF')) and (res[y,x] < 0.1):            
                            cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(255, 0, 0), 3, 8, 0)
                        elif (index < methods.index('cv2.TM_SQDIFF')) and (res[y,x] > 0.95):
                            cv2.rectangle(res_img, (x - int(w/2), y - int(h/2)), (x + int(w/2), y + int(h/2)),(255, 0, 0), 3, 8, 0)
                            #print((x - int(w/2), y - int(h/2)))
                            #print('res[y,x]',res[y,x])
            #print()
            res_img = cv2.resize(res_img, out_size)*255
            show_img = np.copy(img)
            
            if flage:
                for match_loc_t in match_locs:
                    match_loc = match_loc_t[0]
                    right_bottom = (match_loc[0] + template.shape[1], match_loc[1] + template.shape[0])
                    cv2.rectangle(show_img, match_loc, right_bottom, (255,50,0), 3, 6, 0 )
                    cv2.rectangle(res_img, match_loc, right_bottom, (255,50,0), 3, 6, 0 )
            
            cv2.rectangle(res_img, top_left,  bottom_right, (0, 0, 255), 3, 8, 0)
            cv2.putText(res_img, str(index) + method, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, 8, 0)
            cv2.rectangle(show_img,top_left, bottom_right, (0, 0, 255), 2)
            show_img = np.hstack((show_img, res_img)) # 在右邊水平並排顯示視窗
                   
cap.release()
out.release()
cv2.destroyAllWindows()

(47, 38, 3)
38 47


In [216]:
methods.index('cv2.TM_SQDIFF')

4